In [1]:
import zipfile
import os


zip_file_path = '/content/bank-data.zip'
extracted_dir = '/content/bank_data'


with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_dir)


os.listdir(extracted_dir)


['__MACOSX', 'bank-data']

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
dataset_path = '/content/bank_data/bank-data/bank-full.csv'
data = pd.read_csv(dataset_path, sep=';')


data.head()


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [3]:

column_names = data.columns.tolist()

# Print the column names
print("Column Names:")
print(column_names)


Column Names:
['age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'y']


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


data['y'] = data['y'].map({'yes': 1, 'no': 0})


categorical_columns = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome']
data = pd.get_dummies(data, columns=categorical_columns, drop_first=True)


X = data.drop('y', axis=1)
y = data['y']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [5]:
from sklearn.svm import SVC

# Initialize and train the SVM model
svm_model = SVC(kernel='rbf', C=1, gamma='scale', random_state=42)
svm_model.fit(X_train_scaled, y_train)


SVC(C=1, random_state=42)

In [8]:
import numpy as np

# Example input (raw)
new_data = {
    'age': 40,
    'job': 'technician',
    'marital': 'single',
    'education': 'tertiary',
    'default': 'no',
    'balance': 200,
    'housing': 'yes',
    'loan': 'no',
    'contact': 'cellular',
    'day': 15,
    'month': 'may',
    'duration': 300,
    'campaign': 2,
    'pdays': -1,
    'previous': 0,
    'poutcome': 'unknown'
}

# Convert input to a DataFrame
new_data_df = pd.DataFrame([new_data])

# Perform one-hot encoding (ensure the same columns as the training set)
new_data_encoded = pd.get_dummies(new_data_df, columns=categorical_columns)

# Align with the training data columns (add missing columns as 0s)
missing_cols = set(X.columns) - set(new_data_encoded.columns)
for col in missing_cols:
    new_data_encoded[col] = 0
new_data_encoded = new_data_encoded[X.columns]  # Ensure correct column order

# Scale the data
new_data_scaled = scaler.transform(new_data_encoded)


In [10]:
# Predict using the trained model
prediction = svm_model.predict(new_data_scaled)

# Interpret the result
if prediction[0] == 1:
    print("The customer is going to subscribe to a term deposit.")
else:
    print("The customer isn't going to subscribe to a term deposit.")


The customer isn't going to subscribe to a term deposit.
